Making the backend server


In [ ]:
import os
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

Then, we create an instance of a Flask app which just one line of code:

In [ ]:
app = Flask(__name__)

We’ll add more to the Flask app later. For now, let’s load our Machine Learning model that we’ve already saved. We call the following functions and load the model like this:
We'll call our model: --need to complete --

In [ ]:
from keras.models import load_model
from keras.backend import set_session
from skimage.transform import resize
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

print("Loading model")

global sess
sess = tf.Session()
set_session(sess)
global model
model = load_model('our model -- update')
global graph
graph = tf.get_default_graph()

Now that we’ve got our model running, let’s add to our Flask web app. The first thing we want to do is state what the web application will do when it hits a certain URL. To define that, we use the@app.route() function. For my main page, I will use the default URL (i.e. I won’t have any additional “suffixes” to my URL) and so my first parameter will be '/'. An user will interact with my main page in two ways:

1) Load the web page from the server into his browser
2) Upload his image and send it to the server

This requires two different methods of interacting with this page: Interaction #1 will use “GET” (a method primarily for an user to request data from a resource), while interaction #2 will use “POST” (a method primarily for an user to send data to a server to create or update a resource). The code block below therefore says that if I’m at interaction #2, save the file in my “uploads” folder and go to the “prediction” page (and pass in the parameter filename). If not, I’m at interaction #1 and just render the webpage “index.html”.

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def main_page():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join('uploads', filename))
        return redirect(url_for('prediction', filename=filename))
    return render_template('index.html')

Now, we have to create a “prediction” page to go to, and define what the Web app should do when an user is at that page. We use the @app.route()function again, and say that if an user is at the /prediction/<filename> subdirectory, pass in the <filename> into the Python function that I define for that page.

In this prediction page, We will:

1) Read the image based on the filename, and store it as my_image
2) Using the model, predict the probabilities that the image falls into the various classes, and put that under the variable probabilities
3) Find the label and the probability of the top three most probable classes, and put them under predictions
4) Load the template of my webpage predict.html and give them the predictions in Step 3.

The five steps are reflected below:

In [ ]:
@app.route('/prediction/<filename>')
def prediction(filename):
    #Step 1
    my_image = plt.imread(os.path.join('uploads', filename))
    #Step 2
    #my_image_re = resize(my_image, (32,32,3))

    #Step 3
    with graph.as_default():
      set_session(sess)
      probabilities = model.predict(np.array( [my_image,] ))[0,:]
      print(probabilities)
#Step 4
      number_to_class = open("food101_to_csv_values_translation", "r")
      index = np.argsort(probabilities)
      predictions = {
        "class1":number_to_class[index[9]],
        "class2":number_to_class[index[8]],
        "class3":number_to_class[index[7]],
        "prob1":probabilities[index[9]],
        "prob2":probabilities[index[8]],
        "prob3":probabilities[index[7]],
      }
#Step 5
    return render_template('predict.html', predictions=predictions)

All this should be pretty familiar, if you need a refresher, please visit the tutorial:

Lastly, we run the app with port 80, which is the standard port for accessing web pages. Type this at the end of your Python file:

In [1]:
app.run(host='0.0.0.0', port=80)
#Dana website / Or use loackhost

NameError: name 'app' is not defined

Now we have the backbones of our Web application. You might have noticed that we are still missing a few different pieces though. In particular, we’ve called on the HTML web pages like index.html and predict.html but we’ve not built them yet! We will do so in th